Peut être lent au départ car l'hébergement est sur Heroku en "low cost" à mon avis

The API is limited to 5 calls/minutes. 

If you try more, your endpoint will throw back an error.

Les features du CSV (https://lead-program-assets.s3.eu-west-3.amazonaws.com/M05-Projects/fraudTest.csv)
* ,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
* 0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
* 1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.436,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450497999999996,-109.960431,0

vide;
trans_date_trans_time;  # ! N APPARAIT PAS DANS LES DONNEES FOURNIES PAR L'API
cc_num;
merchant;
category;
amt;
first;
last;
gender;
street;
city;
state;
zip;
lat;
long;
city_pop;
job;
dob;
trans_num;
unix_time; # ! APPARAIT EN PLUS DANS LES DONNEES FOURNIES PAR L'API
merch_lat;
merch_long;
is_fraud


In [1]:

import json
import requests
import pandas as pd
from datetime import datetime, timezone

url = "https://real-time-payments-api.herokuapp.com/current-transactions"

response = requests.get(url)
data = response.json()

if isinstance(data, str):
    data = json.loads(data)

columns = data['columns']
index = data['index']
rows = data['data']

df = pd.DataFrame(data=rows, index=index, columns=columns)

pd.set_option('display.max_columns', None)
# display(df)

# ! 17 is hard coded 
col = df['current_time']
df.insert(17, "unix_time", col)

# convert to date string
df.rename(columns={'current_time': 'trans_date_trans_time'}, inplace=True)
timestamp = df["trans_date_trans_time"].iloc[0]
date = datetime.fromtimestamp(timestamp / 1000, tz=timezone.utc)
str_date = date.strftime('%Y-%m-%d %H:%M:%S')

df['trans_date_trans_time'] = df['trans_date_trans_time'].astype(str)
df.at[index[0], "trans_date_trans_time"] = str_date

# reorder columns
cols = df.columns.tolist()
reordered_cols = [cols[-1]] + cols[:-1]
df = df[reordered_cols]

display(df)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
188413,2024-07-08 12:53:40,3587960728692500,fraud_Fisher-Schowalter,shopping_net,27.12,Kathy,Hughes,F,02110 Lucas Freeway Suite 517,Battle Creek,IA,51006,42.3327,-95.6045,1075,"Teacher, early years/pre",1997-01-02,c79331ae7e816402e9ca73b59f1c9d27,1720443220314,42.895369,-95.905408,0
